In [2]:
from py2neo import authenticate, Graph, Relationship
graph = Graph("http://localhost:7474/db/data/")

In [ ]:
index = """
CREATE INDEX ON :AirportDay(airport);
CREATE INDEX ON :Airport(continent);
CREATE INDEX ON :Airport(country);
CREATE INDEX ON :Airport(region);
CREATE INDEX ON :Airport(city);

CREATE INDEX ON :AirportDay(day);
CREATE INDEX ON :AirportDay(airport)
"""

for statement in index.split(";"):
    graph.run(statement)

In [ ]:
%%time
query = """
MATCH (: Airport {country : 'IE'} ) <- [:HAS_AIRPORT] - (ad1:AirportDay)
WITH ad1
MATCH (ad1) - [f1:FLIGHT] - > (ad2:AirportDay)  - [:HAS_AIRPORT] -> (airport2: Airport {continent : 'EU'} ),
(ad2) - [nm:NEXT_MARKET {days: 4}] - (ad3),
(ad3) - [f2:FLIGHT] -> (ad4) - [:HAS_AIRPORT] -> (: Airport {country : 'IE'} )
RETURN f1,f2
"""

t = list(graph.run(query))
print len(t)

In [ ]:
%%time
query = """
MATCH (: Airport {country : 'IE'} ) <- [:HAS_AIRPORT] - (ad1:AirportDay)
WITH ad1
MATCH (ad1) - [f1:FLIGHT] - > (ad2:AirportDay)  - [:HAS_AIRPORT] -> (: Airport) - [IN_CONTINENT] -(:Continent {continent: 'EU}) ,
(ad2) - [nm:NEXT_MARKET {days: 4}] - (ad3),
(ad3) - [f2:FLIGHT] -> (ad4) - [:HAS_AIRPORT] -> (: Airport {country : 'IE'} )
RETURN f1,f2
"""

t = list(graph.run(query))
print len(t)

In [8]:
%%time
query = """
MATCH (airport1: Airport {airport : 'Dublin Airport'} ) <- [:HAS_AIRPORT] - (ad1:AirportDay)
WITH ad1, airport1
MATCH (ad1) - [f1:FLIGHT] - > (ad2:AirportDay)  - [:HAS_AIRPORT] -> (airport2: Airport {continent : 'EU'} )
WITH f1, airport1, ad2
MATCH (ad2) - [nm:NEXT_MARKET {days: 4}] - (ad3)
WITH f1, airport1, ad3
MATCH (ad3) - [f2:FLIGHT] -> (ad4) - [:HAS_AIRPORT] -> (airport1)
RETURN f1,f2
"""

t = list(graph.run(query))
print len(t)

1520
Wall time: 12.4 s
